In [ ]:
#Tahap 2
#Capture Dataset per-frame
import cv2
import mediapipe as mp
import os

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

label = "A" 
save_path = f"BISINDO/{label}"
os.makedirs(save_path, exist_ok=True)

cap = cv2.VideoCapture(0)
counter = 0

while True:
    ret, frame = cap.read()
    if not ret: break
    img = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    if res.multi_hand_landmarks:
        for hand in res.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand, mp_hands.HAND_CONNECTIONS)

        filename = f"{save_path}/{label}_{counter}.jpg"
        cv2.imwrite(filename, img)
        counter += 1

    cv2.putText(img, f"Saved: {counter}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Capture Dataset BISINDO", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#Pembuatan Metadata Dataset
import csv
import os
from datetime import datetime

dataset_dir = "BISINDO"
metadata_file = "dataset_metadata.csv"

with open(metadata_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["filename","label","user_id","lighting","background","distance","angle","timestamp"])

    for label in os.listdir(dataset_dir):
        path = os.path.join(dataset_dir, label)
        for file in os.listdir(path):
            writer.writerow([
                file,
                label,
                "user1",
                "default",
                "plain",
                "medium",
                "front",
                datetime.now().isoformat()
            ])


In [ ]:
#Tahap 3
#Deteksi Tangan + Crop ROI
import cv2
import mediapipe as mp
import os

source_dir = "BISINDO/A"
dest_dir = "BISINDO_PRE/A"
os.makedirs(dest_dir, exist_ok=True)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True)
mp_draw = mp.solutions.drawing_utils

def crop_hand(img, landmarks):
    h, w, _ = img.shape
    xs = [lm.x*w for lm in landmarks.landmark]
    ys = [lm.y*h for lm in landmarks.landmark]

    x1, y1 = int(min(xs))-20, int(min(ys))-20
    x2, y2 = int(max(xs))+20, int(max(ys))+20

    return img[y1:y2, x1:x2]

for file in os.listdir(source_dir):
    img = cv2.imread(os.path.join(source_dir, file))
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    if res.multi_hand_landmarks:
        hand = res.multi_hand_landmarks[0]
        roi = crop_hand(img, hand)
        cv2.imwrite(os.path.join(dest_dir, file), roi)


In [ ]:
#Resize & Normalisasi
import cv2
import os

src = "BISINDO_PRE/A"
dst = "BISINDO_PRE_FINAL/A"
os.makedirs(dst, exist_ok=True)

IMAGE_SIZE = 128

for file in os.listdir(src):
    img = cv2.imread(os.path.join(src, file))
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = img / 255.0  # normalisasi

    # simpan sebagai numpy
    import numpy as np
    np.save(os.path.join(dst, file.replace(".jpg", ".npy")), img)


In [ ]:
#Augmentasi
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2

datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.2,
    brightness_range=[0.7, 1.3],
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

src = "BISINDO_PRE_FINAL/A"
dst = "BISINDO_AUG/A"
os.makedirs(dst, exist_ok=True)

for file in os.listdir(src):
    img = np.load(os.path.join(src, file))
    img = img.reshape((1,128,128,3))

    gen = datagen.flow(img, batch_size=1)
    for i in range(5):
        batch = gen.next()[0]
        cv2.imwrite(f"{dst}/{file}_{i}.jpg", batch*255)


In [ ]:
#Tahap 4
#Pembuatan Model CNN (Statis)
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input((128,128,3)),
    
    layers.Conv2D(32,3,activation="relu"),
    layers.MaxPool2D(),
    
    layers.Conv2D(64,3,activation="relu"),
    layers.MaxPool2D(),
    
    layers.Conv2D(128,3,activation="relu"),
    layers.MaxPool2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(26, activation="softmax")  # 26 huruf
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


In [ ]:
#Training Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    batch_size=32
)
model.save("bisindo_cnn.h5")

In [ ]:
#Tahap 5
#Pipeline Real-time
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model("bisindo_cnn.h5")

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break

    img = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    if res.multi_hand_landmarks:
        hand = res.multi_hand_landmarks[0]

        # crop tangan
        h, w, _ = img.shape
        xs = [lm.x*w for lm in hand.landmark]
        ys = [lm.y*h for lm in hand.landmark]

        x1,y1 = int(min(xs))-20, int(min(ys))-20
        x2,y2 = int(max(xs))+20, int(max(ys))+20

        roi = img[y1:y2, x1:x2]

        if roi.size > 0:
            roi = cv2.resize(roi, (128,128)) / 255.0
            roi = np.expand_dims(roi, axis=0)

            pred = model.predict(roi)
            label = chr(np.argmax(pred) + 65)   # 0→A, 1→B, ...

            cv2.putText(img, f"Prediksi: {label}", (10,50),
                        cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

    cv2.imshow("BISINDO Interpreter", img)
    if cv2.waitKey(1) == ord('q'): break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#Tahap 6
#Output Teks dan Suara
from gtts import gTTS
import os

pred_text = "A"
tts = gTTS(pred_text, lang="id")
tts.save("output.mp3")
os.system("start output.mp3")


In [ ]:
#Tahap 7 evaluasi
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = np.argmax(y_test,axis=1)
y_pred = np.argmax(model.predict(x_test),axis=1)

print(classification_report(y_true,y_pred))
print(confusion_matrix(y_true,y_pred))
